Analyze Pressure Data (find traces where pressure significantly changes during the trace)

In [ ]:
# Find traces where the OV_1_WT_1_Pneumatic_System_Pressure shows a significant change.
abs_range_threshold = 0.5
abs_range_from_zero = 0.6

sql = f"""
WITH vals AS (
    SELECT
        "trace:SubProcessID",
        TRY_CAST("stream:value" AS DOUBLE) AS v,
        "stream:timestamp"
    FROM sensor_data
    WHERE "stream:observation" = 'http://iot.uni-trier.de/FTOnto#OV_1_WT_1_Pneumatic_System_Pressure'
        AND "org:resource" = 'ov_1'
        AND TRY_CAST("stream:value" AS DOUBLE) IS NOT NULL
),
diffs AS (
    SELECT
        "trace:SubProcessID",
        v,
        LAG(v) OVER (PARTITION BY "trace:SubProcessID" ORDER BY "stream:timestamp") AS prev_v
    FROM vals
)
SELECT
    d."trace:SubProcessID" AS trace_id,
    COUNT(*) AS n_samples,
    MIN(v) AS v_min,
    MAX(v) AS v_max,
    (MAX(v) - MIN(v)) AS v_range,
    AVG(v) AS v_mean,
    ABS(AVG(v)) AS mean_abs_diff0,
    GREATEST(ABS(MIN(v)), ABS(MAX(v))) AS max_abs_from_zero
FROM diffs d
GROUP BY d."trace:SubProcessID"
HAVING (MAX(v) - MIN(v)) > {abs_range_threshold}
    OR GREATEST(ABS(MIN(v)), ABS(MAX(v))) > {abs_range_from_zero}
    OR MIN(v) < 0
ORDER BY v_range DESC
"""

df_sensor_grouped = con.execute(sql).df()

# ------------------------

pd.set_option('display.max_colwidth', None)
pd.set_option('display.width', None)
pd.set_option('display.max_columns', None)
display(df_sensor_grouped)